In [ ]:
import json
import tweepy
from datetime import datetime, timezone
from kafka import KafkaProducer

In [ ]:
consumer_key    = 'T3bxHQI78bQSMxhryPQk4rgMs'
consumer_secret = 'uZEP6eLjUGGkS4YtRvm3PISnpMlho7vniHKeoZMF2NVu9B6rpe'
access_token    = '1207006565074640897-c761jqaeRPkTPeyjBFxlPqsirDhYQ1'
access_secret   = 'qmJiaHbct4hshX4mcVIx2BuKCPdo8xndNPb7bOLes4MOs'

In [ ]:
# Probably you can compare the performance of opposing hashtags insuch way
#Keywords = "#OwenJonesIsAWankerDay #SolidarityWithOwenJonesDay"

In [ ]:
Keywords = 'Israel'

In [ ]:
class TwitterStreamListener(tweepy.StreamListener):
    def __init__(self):
        self.producer = KafkaProducer(bootstrap_servers='localhost:9092',
                                      key_serializer=str.encode, 
                                      value_serializer=lambda v: json.dumps(v).encode('utf-8'))

        self.datefmt = "%a %b %d %H:%M:%S %z %Y"
        self.topic = "TweeterArchive"
        self.tweets = []
        self.rawdata = [] 
        self.count = 0
    def on_data(self, data):
        # data is the full *tweet* json data
        api_events = json.loads(data)
        
        #filter out retweets
        if not api_events['text'].startswith('RT'):            
            
            # Gathering relevant values
            # Tweet-related values
            event_keys = ['created_at', 'id', 'text', 'geo', 'coordinates', 'place',
                          "quote_count", "reply_count", "retweet_count", "favorite_count" ]
            twitter_events = {k:v for k,v in api_events.items() 
                              if k in event_keys}                    
            
            # When tweets are longer than 140 symbols, the 'text' field gets truncated, and the
            # full text is in ['extended_tweet']['full_text']
            if 'extended_tweet' in api_events:
                twitter_events['text'] = api_events['extended_tweet']['full_text']

            # User-related values
            user_keys = ['id', 'name', 'screen_name','created_at', 'location', 'url',
                         'protected', 'verified', 'followers_count', 'friends_count',
                         'listed_count', 'favourites_count', 'statuses_count', 'withheld_in_countries']
            user_events = {k:v for k,v in api_events['user'].items() 
                           if k in user_keys}

            events = {'user':user_events, 'tweet': twitter_events }
    
            key = self.date_fmt_convert(twitter_events['created_at'])
            
            # save raw json data in memory array for debugging
            self.rawdata.append(data)
            # save processed dictionary in memory array for debugging
            self.tweets.append(events)
            # print last tweet to console on the same line
            ltweet = events['tweet']['text'][:50].replace('\n','')
            print('\r tweets received: '+ str(self.count) + '| last tweet: ' + ltweet,end='')
             
            self.send_to_kafka(key, events)                     
        self.count+=1        

    def send_to_kafka(self, key, data):  
        
        # send data to kafka topic(s)
        #self.producer.send('TweeterArchive', events)
        #self.producer.send('TweeterData', events) 
        self.producer.send(self.topic, key = key, value = data)
        self.producer.flush()
        
    def date_fmt_convert(self, date):        
        # Convert twitter datetime format such as 
        # "Sat Jan 4 11:39:13 +0500 2019"
        # to partitioning-compatible format and shift it to utc, so: 
        # '2019-01-04_06-39'
        date_time_obj = datetime.strptime(date, self.datefmt)
        date_time_obj = date_time_obj.astimezone(tz=timezone.utc)
        return date_time_obj.strftime("%Y-%m-%d_%H-%M")

 
    def on_error(self, status_code):
        if status_code == 420:
            return False

In [ ]:
def initialize():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    stream = TwitterStreamListener()
    twitter_stream = tweepy.Stream(auth=api.auth, listener=stream)
    twitter_stream.filter(track=[Keywords], languages=['en']) # -filter:retweets

In [ ]:
initialize()

## Sandbox 
below are cells that I played with to test portions of code


In [ ]:
wat = stream.tweets

In [ ]:
wat

In [ ]:
len(wat)
wat4 = wat[20]
i=0

In [ ]:
i+=1
print(json.loads(wat[i])['text'])

print('-'*100)
print(json.loads(wat[i])['extended_tweet']['full_text'])


In [ ]:
wat[i]

In [ ]:
print(json.loads(wat[i])['extended_tweet']['full_text'])

In [ ]:
print(json.dumps(json.loads(wat[i])))

In [ ]:
wat4

In [ ]:
api_event = json.loads(wat4)

In [ ]:
api_event['retweeted']

In [ ]:
api_event

In [ ]:
str_event = json.dumps(api_event, ensure_ascii=False)
str_event

In [ ]:
with open('./test.json','w') as f:
    str_event = json.dump(api_event,f, ensure_ascii=False)
str_event

In [ ]:
print(bytes(str_event, 'utf-8' ))

In [ ]:
wat[4]['retweeted_status']#['extended_tweet']['full_text']